In [1]:
import numpy as np
import matplotlib.pyplot as plt
import gymnasium as gym
import math
import mlflow
import optuna
import os

from TMDP import TMDP
from algorithms import *
from model_functions import *
from policy_utils import *
from experiment_result_utils import *
from constants import *

from FrozenLake import *
from CurriculumPPO import CurriculumPPO

In [2]:
is_slippery = False
reward_shape = True
dense_reward = True
num_bins = 15
nrows = 30

num_runs = 10
episodes = 4900000
checkpoint_step=500
test_episodes = 1000

shape_range=(-1,0)
goal_reward = 1.
debug = False
param_decay=True

lam = 1
experiment_results = []
tests_returns = []

In [3]:
#Frozen Lake Environment
tau = 0.6
nS = nrows**2
nA = 4
gamma = 0.999
xi = np.ones(nS) * 1/nS

In [4]:
model_lr = 0.373715
pol_lr = 0.215723
temp = 36.039501
epochs= 3
entropy_coef = 0.400000

batch_size = 20
final_temp=1

eps_ppo = 0.2
eps_model = compute_eps_model(gamma, tau, episodes/10)


In [5]:
run_name = f"CurrPPO_{num_bins}"

mlflow.set_tracking_uri(MLFLOW_URI)
experiment_name = f"FrozenLake_{nrows}x{nrows}_{num_bins}"
experiment_id = get_or_create_experiment(experiment_name)
mlflow.set_experiment(experiment_name)

save_path = f"results/{experiment_name}/run_{run_name}"
label = run_name.split("_")[0]

In [6]:
def run_experiment(index, seed, run_name, change_map=False):
    sub_run_name = f"{run_name}_{index}"
    
    with mlflow.start_run(nested=True, run_name=sub_run_name):
        # Environment specific configuration   
        map_seed = seed if change_map else constants.SEEDS[0]
        set_policy_seed(seed)
        env = FrozenLakeEnv(is_slippery=is_slippery, seed=seed, 
                        desc=generate_random_map(nrows, seed=map_seed), 
                        reward_shape=reward_shape,
                        num_bins=num_bins,
                        dense_reward=dense_reward,
                        shape_range=shape_range,
                        goal_reward=goal_reward,
                        )
        
        # Environment independent configuration
        tmdp = TMDP(env, xi, tau=tau, gamma=gamma, seed=seed)
        tmdp.update_tau(tau)
        curr_PPO = CurriculumPPO(tmdp, checkpoint_step=checkpoint_step)

        curr_PPO.train(model_lr, pol_lr, batch_size=batch_size, 
                lam=lam, temp=temp, final_temp=final_temp,
                episodes=episodes, eps_ppo=eps_ppo,
                eps_model=eps_model,
                param_decay=param_decay, epochs=epochs, max_length=1000,
                entropy_coef=entropy_coef, debug=debug)
        
        
        avg_return = np.average(curr_PPO.reward_records[-10:])/batch_size
        
        
        mlflow.log_metric("Avg Return", avg_return)
        
        run_dict = {
            "episodes": curr_PPO.episodes,
            "model_lr": model_lr,
            "pol_lr": pol_lr,
            "temp": temp,
            "final_temp": final_temp,
            "batch_size": batch_size,
            "epochs": epochs,
            "lam": lam,
            "eps_ppo": eps_ppo,
            "eps_model": eps_model,
        }

        mlflow.log_params(run_dict)
        mlflow.set_tags(tags={
            "run_name": run_name,
            "change_map": change_map,
            "seed": seed,
            "tau": tau,
            "gamma": gamma,
            "checkpoint_step": checkpoint_step,
            "test_episodes": test_episodes,
            "index": index,
            "dense_reward": dense_reward,
            "shape_range": shape_range,
            "goal_reward": goal_reward,
            "reward_shape": reward_shape,
        })


        
        test_policies_return = test_policies(tmdp, curr_PPO.thetas, test_episodes, 1e-5)
        
        result_dict = {
            "thetas" : curr_PPO.thetas,
            "taus" : curr_PPO.taus,
            "reward_records" : curr_PPO.reward_records,
            "test_policies_return" : test_policies_return,
            "index" : index,
        }

        tests_returns.append(test_policies_return)
        experiment_results.append(result_dict)
        # Save artifact to MLFlow
        try:
            save_to_mlflow(result_dict, run_name)
        except Exception as e:
            print(f"Error saving to MLFlow: {e}")
            print("Saving locally instead.")
            time.sleep(5)
            path = save_path+f"/{sub_run_name}"
            save(path, result_dict)

In [7]:
def run_experiments(change_map=False, num_runs=10):
    with mlflow.start_run(run_name=run_name):
        
        for i in range(num_runs):
            seed = constants.SEEDS[i]
            run_experiment(i, seed, run_name, change_map)
        
        experiment_dict = {
            "tests_returns": tests_returns,
            "num_runs": num_runs,
            "change_map": change_map,
            "num_bins": num_bins,
            "label": label,
        }
        try:
            save_to_mlflow(experiment_dict, run_name)
        except Exception as e:
            print(e)
            print("Something went wrong saving the experiment results to MLFlow.")
            print("Saving locally instead.")
            time.sleep(5)
            save(save_path, experiment_dict)

        rewards_fig = plot_avg_test_return(tests_returns, f"CPPO Avg Rewards on {num_runs} runs")
        try:
            mlflow.log_figure(figure=rewards_fig, artifact_file="reward_image.png")
        except Exception as e:
            print(e)
            print("Something went wrong saving the figure to MLFlow.")
            print("Saving locally instead.")
            time.sleep(5)
            rewards_fig.savefig(save_path+"/reward_image.png")

In [8]:
run_experiments(change_map=False, num_runs=num_runs)

Current seed for result reproducibility: 2999
Episode: 5000 reward: -2814.333333333333 length: 53
Episode: 10000 reward: -3512.7999999999997 length: 56
Episode: 15000 reward: -3706.9333333333348 length: 80
Episode: 20000 reward: -1379.6666666666667 length: 34
Episode: 25000 reward: -2434.7333333333327 length: 29
Episode: 30000 reward: -1702.6 length: 97
Episode: 35000 reward: -2657.6 length: 93
Episode: 40000 reward: -3380.2666666666664 length: 109
Episode: 45000 reward: -1901.6666666666667 length: 103
Episode: 50000 reward: -3185.2666666666687 length: 150
Episode: 55000 reward: -1964.8666666666666 length: 43
Episode: 60000 reward: -1450.9999999999998 length: 50
Episode: 65000 reward: -2152.7333333333336 length: 39
Episode: 70000 reward: -294.06666666666666 length: 15
Episode: 75000 reward: -2994.7333333333318 length: 117
Episode: 80000 reward: -3361.7999999999997 length: 81
Episode: 85000 reward: -1918.2000000000003 length: 105
Episode: 90000 reward: -2749.1333333333346 length: 84
Epi

In [ ]:
print(len(tests_returns[0]))

In [ ]:
num = 8
indexes = [0, 1, 2, 3, 4, 5,  ]
indexes = [6,9]
rew = [tests_returns[i] for i in indexes]
len(rew)

In [ ]:
rewards_fig = plot_avg_test_return(rew, f"CPPO Avg Rewards on {num_runs} runs")